In [1]:
import re
from tqdm.notebook import tqdm

# Importing Libraries

In [1]:
import numpy as numpy
import pandas as pd

To visualize the data in the project

In [2]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
import sklearn
from sklearn.utils import shuffle 
from sklearn.feature_extraction.text import TfidfVectorizer

NLP Preprocessing libraries

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [5]:
import re
import random
import gensim

In [6]:
from collections import Counter
import unicodedata as udata
import string

checking the versions 

In [8]:
print(sklearn.__version__)
print(matplotlib.__version__)
print(numpy.__version__)
print(pd.__version__)
print(nltk.__version__)

0.22.2.post1
3.2.2
1.19.5
1.1.5
3.2.5


#File read

In [7]:
import gdown
gdown.download('https://drive.google.com/uc?id=1iW8-Y9aePxyEhq1L7-_ldD9L-TzDd_Qb','Code_mixed_data.txt',quiet=True)

'Code_mixed_data.txt'

In [8]:
fp = open("Code_mixed_data.txt", 'r')

X_train_csnli=[]

tweets = []
tweets_label = []
for line in fp:
  lst = []
  lst = line.strip().split('\t')
  tweets.append(lst[1])
  tweets_label.append(lst[2])

In [ ]:
tweets

In [12]:
len(tweets_label)

3879

# clean_tweet_texts

In [9]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'        # remove @ mentions from tweets
pat2 = r'https?://[^ ]+'        # remove URLs from tweets
combined_pat = r'|'.join((pat1, pat2)) #addition of pat1 and pat2
www_pat = r'www.[^ ]+'         # remove URLs from tweets
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",   # converting words like isn't to is not
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner(text):  # define tweet_cleaner function to clean the tweets
    soup = BeautifulSoup(text, 'lxml')    # create beautiful soup object
    souped = soup.get_text() 
    #print('tweer_clean','\n')
    #print(text,'\n')  # get only text from the tweets 
    #print(souped,'\n')
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")    # remove utf-8-sig code
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed) # calling combined_pat
    #print(stripped,'\n')
    stripped = re.sub(www_pat, '', stripped) #remove URLs
    #print(stripped,'\n')
    lower_case = stripped.lower()      # converting all into lower case
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case) # converting words like isn't to is not
    #print(neg_handled,'\n')
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)       # will replace # by space
    #print(letters_only,'\n')
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1] # Word Punct Tokenize and only consider words whose length is greater than 1
    return (" ".join(words)).strip() # join the words

In [10]:
#Note that we have 1600000 instances. But processing so many instances will take a very very long time.
#Hence, restricting to rather 50000 instances.
limit=len(tweets_label)
import time; 
ms = time.time()
#nums = [0,400000,800000,1200000,1600000] # used for batch processing tweets
#nums = [0, 9999]
clean_tweet_texts = [] # initialize list
for i in range(0,limit): # batch process 1.6 million tweets 
    if i % 2000==0:
        print(i, time.time()-ms)
    clean_tweet_texts.append(tweet_cleaner(tweets[i]))  # call tweet_cleaner function and pass parameter as all the tweets to clean the tweets and append cleaned tweets into clean_tweet_texts list

0 7.700920104980469e-05
2000 0.4889364242553711


In [ ]:
clean_tweet_texts

# Tokenizing tweets

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

tokenize word in clean_tweet_texts and append it to word_tokens list

In [12]:
word_tokens = [] # initialize list for tokens
for word in clean_tweet_texts:  # for each word in clean_tweet_texts
    word_tokens.append(word_tokenize(word)) #tokenize word in clean_tweet_texts and append it to word_tokens list

In [ ]:
word_tokens

# Lemmatizing

In [13]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
df1 = [] # initialize list df1 to store words after lemmatization
from nltk.stem import WordNetLemmatizer # import WordNetLemmatizer from nltk.stem
lemmatizer = WordNetLemmatizer() # create an object of WordNetLemmatizer
for l in word_tokens: # for loop for every tokens in word_token
    b = [lemmatizer.lemmatize(q) for q in l] #for every tokens in word_token lemmatize word and giev it to b
    df1.append(b) #append b to list df1

In [ ]:
df1

In [15]:
clean_df1 =[] # initialize list clean_df1 to join word tokens after lemmatization
for c in df1:  # for loop for each list in df1
    a = " ".join(c) # join words in list with space in between and give it to a
    clean_df1.append(a) # append a to clean_df1

#Train Test split

In [16]:
from sklearn.model_selection  import train_test_split #from sklearn.cross_validation import train_test_split to split the data into training and tesing set
X_train, X_test, y_train, y_test = train_test_split(tweets, tweets_label, test_size = 0.20, random_state= 1) # split the data into traing and testing set where ratio is 80:20

In [ ]:
X_train

In [ ]:
pip install flair

In [24]:
from flair.data import Sentence

X_train_flair = [Sentence(tweet) for tweet in X_train]

In [25]:
from flair.data import Sentence

X_test_flair = [Sentence(tweet) for tweet in X_test]

In [32]:
X_test_flair

[Sentence: "rafu apko nhi bolna already wo dechyka hota h"   [− Tokens: 9],
 Sentence: "Handsom to hai mera bhai bhai he mera"   [− Tokens: 8],
 Sentence: "Baat kidhr hogi"   [− Tokens: 3],
 Sentence: "advanced eid mubarak"   [− Tokens: 3],
 Sentence: "Bhai shaadi b kroge ya randwe hi mroge"   [− Tokens: 8],
 Sentence: "Yr tum sab ko bahut takleef he bhai ki sadi . .. se . .. heiiiiinn ?"   [− Tokens: 17],
 Sentence: "modi ji apko sahi dimag ko me salute karta hu"   [− Tokens: 10],
 Sentence: "Hi bizy man ap hum sy kaha baat karo gy apko razan Mubarak hooo eid par SMG zaror karna plz plz bye"   [− Tokens: 22],
 Sentence: "Assalaam waalaikum bhaijaan ..."   [− Tokens: 4],
 Sentence: "Pagalo aa gae bus chalo ghus jao bus me jiyada aur rahy yaha to phir apka koi ilaj nhi"   [− Tokens: 19],
 Sentence: "5000comment in 5minute"   [− Tokens: 3],
 Sentence: "BAJRANGI BHAIJAAN 700 CRORE ! !"   [− Tokens: 6],
 Sentence: "Sir meri ek hi ichha h apse milna"   [− Tokens: 8],
 Sentence: "Mjh say bat

In [27]:
from flair.embeddings import BertEmbeddings
import numpy as np
embedding_bert = BertEmbeddings("bert-base-cased")

X_train_bert = []
for sent in X_train_flair:
  sent_emb = np.array(embedding_bert.embed(sent)[0])
  emb = []
  for token in sent_emb:
    emb.append(np.array(token.embedding.cpu()))
  X_train_bert.append(np.array(emb))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  This is separate from the ipykernel package so we can avoid doing imports until


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
EMBEDDINGS_SIZE = X_train_bert[0].shape[1]
EMBEDDINGS_SIZE

3072

In [37]:
del X_test_flair[629:630]

In [38]:
del y_test[629:630]

In [ ]:
X_test_bert = []
i=0
for sent in X_test_flair:
  print(sent,i)
  print('\n')
  i=i+1
  sent_emb = np.array(embedding_bert.embed(sent)[0])
  emb = []
  for token in sent_emb:
    emb.append(np.array(token.embedding.cpu()))
  X_test_bert.append(np.array(emb))

In [40]:
MAX_SEQ_LEN_train = max([len(tweet.split()) for tweet in X_train])
print(MAX_SEQ_LEN_train)

MAX_SEQ_LEN_test = max([len(tweet.split()) for tweet in X_test])
print(MAX_SEQ_LEN_test)

MAX_SEQ_LEN = min(MAX_SEQ_LEN_train, MAX_SEQ_LEN_test)
MAX_SEQ_LEN

49
48


48

In [41]:
np.save('test_bert', X_test_bert)
np.save('train_bert',X_train_bert)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
#import numpy as np
#X_train_bert = np.load("train_bert.npy", allow_pickle=True)
#X_test_bert = np.load("test_bert.npy", allow_pickle=True)

In [42]:
import torch
from torch.nn.utils.rnn import pad_sequence

emb_list = []
for i in X_train_bert:
  emb_list.append(torch.tensor(i[:48]))
X_train_bert_padded = pad_sequence(emb_list, batch_first=True)

emb_list = []
for i in X_test_bert:
  emb_list.append(torch.tensor(i[:MAX_SEQ_LEN]))
X_test_bert_padded = pad_sequence(emb_list, batch_first=True)

In [45]:
y_dct = {'0':0, '1':1,'2':2}
print(y_train[:5])

for i, sentiment in enumerate(y_train):
  y_train[i] = y_dct[sentiment]

import numpy as np 
y_train = np.array(y_train)
print(y_train[:5])

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train.reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()

print(y_train[:5])

['2', '1', '1', '2', '0']
[2 1 1 2 0]
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [46]:
y_dct = {'0':0, '1':1,'2':2}
print(y_test[:5])

y_test_int = []
for i, sentiment in enumerate(y_test):
  y_test_int.append(y_dct[sentiment])

import numpy as np 
y_test_int = np.array(y_test_int)
print(y_test_int[:5])

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_test_int.reshape(-1, 1))
y_test= enc.transform(y_test_int.reshape(-1, 1)).toarray()

print(y_test[:5])

['0', '2', '1', '2', '0']
[0 2 1 2 0]
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


#Bi LSTM

In [48]:

!pip install keras_Self_attention
!pip install keras_tqdm

  Created wheel for keras-Self-attention: filename=keras_self_attention-0.50.0-py3-none-any.whl size=19414 sha256=7da0b788c0b903323dbfff370af3d31de9df43d1d63887c2caf95abbee84853e
  Stored in directory: /root/.cache/pip/wheels/92/7a/a3/231bef5803298e7ec1815215bc0613239cb1e9c03c57b13c14
Successfully built keras-Self-attention


In [49]:
import pickle

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Bidirectional
from keras import optimizers
from keras.layers.recurrent import LSTM
import keras
from keras.layers import Dropout
from keras_self_attention import SeqSelfAttention
from keras_tqdm import TQDMNotebookCallback

In [51]:
#------------------------------------MODELS------------------------------------------#
 
"""#### Basic BiLSTM, no appends
**MODEL 0.0**:
*   BERT
*   2 LSTM layer
*   4 dense
*   2 dense
"""

model_Bi_LSTM_1 = Sequential()
model_Bi_LSTM_1.add(Bidirectional(LSTM(10, dropout=0.5, recurrent_dropout=0.5, return_sequences=True), 
                                  input_shape=(48, 3072), 
                                  merge_mode='concat'))
model_Bi_LSTM_1.add(Bidirectional(LSTM(10, dropout=0.5, recurrent_dropout=0.5), merge_mode='concat'))
model_Bi_LSTM_1.add(Dense(5, activation='softmax'))
model_Bi_LSTM_1.add(Dense(3, activation='softmax'))
model_Bi_LSTM_1.summary()
model_Bi_LSTM_1.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_Bi_LSTM_1.fit(x=X_train_bert_padded.cpu().numpy(), 
                    y=y_train, 
                    validation_data=(X_test_bert_padded.cpu().numpy(), y_test),	
                    batch_size=128, 
                    epochs=10, 
                    shuffle=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 48, 20)           246640    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 20)               2480      
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 5)                 105       
                                                                 
 dense_3 (Dense)             (None, 3)                 18        
                                                                 
Total params: 249,243
Trainable params: 249,243
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
25/25 [==============================

In [ ]:
#from keras.models import load_model
#model_Bi_LSTM_1.save('my_model_Bi_LSTM_1.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model
# returns a compiled model
# identical to the previous one
#model_Bi_LSTM_1 = load_model('my_model_Bi_LSTM_1.h5')


In [52]:
model_Bi_LSTM_1.evaluate(X_test_bert_padded.cpu().numpy(), y_test)[1]

25/25 [==============================] - 3s 109ms/step - loss: 0.8747 - accuracy: 0.6206


0.6206451654434204

In [62]:
y_pred = []          

for tweet in X_test_bert_padded.cpu().numpy():
  y_pred.append(model_Bi_LSTM_1.predict(np.array([tweet])))

#y_pred = np.array([i[0] for i in y_pred])

In [ ]:
y_list= list(y_pred)

In [91]:
def maximum(a, b, c):
    if (a >= b) and (a >= c):
        largest = 0
    elif (b >= a) and (b >= c):
        largest = 1
    else:
        largest = 2
    return largest

In [92]:
y_pred_int =[]
for i in range(len(y_list)):
  y_pred_int.append(maximum((list(list(y_list[i]))[0][0]),(list(list(y_list[i]))[0][1]),(list(list(y_list[i]))[0][2])))


In [93]:
y_pred = np.array(y_pred_int)

In [94]:
y_pred

array([1, 1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1,
       0, 2, 1, 1, 0, 1, 2, 0, 1, 0, 1, 1, 0, 2, 0, 1, 0, 2, 2, 0, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 0, 1, 1, 1,
       1, 2, 1, 2, 1, 2, 0, 2, 0, 0, 2, 1, 0, 2, 1, 2, 2, 1, 1, 2, 2, 1,
       2, 1, 2, 0, 0, 1, 1, 0, 0, 2, 1, 0, 2, 0, 0, 1, 1, 2, 1, 1, 1, 2,
       1, 2, 1, 2, 0, 0, 2, 1, 2, 1, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 2, 2, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 1, 2, 0, 2, 2,
       2, 1, 1, 1, 0, 0, 2, 1, 1, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0, 2, 0, 1,
       1, 2, 2, 2, 0, 1, 1, 1, 1, 0, 2, 1, 0, 1, 2, 1, 1, 2, 1, 1, 2, 0,
       1, 2, 1, 0, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 0, 1, 1, 2,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 2,
       1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1,

In [95]:
from sklearn.metrics import classification_report as clf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd

target_names=['negative','neutral','positive']

df_cm = pd.DataFrame(confusion_matrix(y_test_int, y_pred), columns=target_names, index=target_names)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
print(df_cm)
print("\n")
print("Accuracy: ",accuracy_score(y_test_int, y_pred))
print("\n","Report:")
print(clf(y_test_int, y_pred, target_names=target_names))

Predicted  negative  neutral  positive
Actual                                
negative         59       46        14
neutral          46      283        56
positive         39       93       139


Accuracy:  0.6206451612903225

 Report:
              precision    recall  f1-score   support

    negative       0.41      0.50      0.45       119
     neutral       0.67      0.74      0.70       385
    positive       0.67      0.51      0.58       271

    accuracy                           0.62       775
   macro avg       0.58      0.58      0.58       775
weighted avg       0.63      0.62      0.62       775



#Bi LSTM with Self attention

In [96]:
"""
#### Self Attention Library, no appends
**MODEL 1.1 SelfAtt**
"""

model_Bi_LSTM_att1 = Sequential()
model_Bi_LSTM_att1.add(Bidirectional(LSTM(10, dropout=0.5, recurrent_dropout=0.5, return_sequences=True), input_shape=(48, 3072), merge_mode='concat'))
model_Bi_LSTM_att1.add(SeqSelfAttention(attention_activation='sigmoid'))
model_Bi_LSTM_att1.add(Bidirectional(LSTM(5, dropout=0.5, recurrent_dropout=0.5), merge_mode='concat'))
#model_Bi_LSTM_att1.add(Dense(5, activation='softmax'))
model_Bi_LSTM_att1.add(Dense(3, activation='softmax'))
model_Bi_LSTM_att1.summary()
model_Bi_LSTM_att1.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_Bi_LSTM_att1.fit(x=X_train_bert_padded.cpu().numpy(), 
                       y=y_train, 
                       validation_data=(X_test_bert_padded.cpu().numpy(), y_test),	
                       batch_size=64, 
                       epochs=10, 
                       shuffle=True)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (None, 48, 20)           246640    
 nal)                                                            
                                                                 
 seq_self_attention (SeqSelf  (None, 48, 20)           1345      
 Attention)                                                      
                                                                 
 bidirectional_5 (Bidirectio  (None, 10)               1040      
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 3)                 33        
                                                                 
Total params: 249,058
Trainable params: 249,058
Non-trainable params: 0
________________________________________________

In [ ]:
#from keras.models import load_model
#model_Bi_LSTM_att1.save('my_model_Bi_LSTM_att1.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model
# returns a compiled model
# identical to the previous one
#model_Bi_LSTM_att1 = load_model('my_model_Bi_LSTM_att1.h5')


In [97]:
model_Bi_LSTM_att1.evaluate(X_test_bert_padded.cpu().numpy(), y_test)[1]

25/25 [==============================] - 3s 114ms/step - loss: 0.9036 - accuracy: 0.6400


0.6399999856948853

In [99]:
y_pred = []          

for tweet in X_test_bert_padded.cpu().numpy():
  y_pred.append(model_Bi_LSTM_att1.predict(np.array([tweet])))

#y_pred = np.array([i[0] for i in y_pred])

In [ ]:
y_pred

In [101]:
y_list= list(y_pred)

In [102]:
y_pred_int =[]
for i in range(len(y_list)):
  y_pred_int.append(maximum((list(list(y_list[i]))[0][0]),(list(list(y_list[i]))[0][1]),(list(list(y_list[i]))[0][2])))


In [103]:
y_pred_int = np.array(y_pred_int)

In [104]:
y_pred_int

array([1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1,
       1, 2, 1, 1, 0, 1, 2, 0, 1, 1, 1, 2, 0, 1, 1, 1, 0, 2, 2, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1,
       1, 2, 1, 2, 1, 1, 0, 2, 1, 1, 1, 1, 0, 2, 1, 1, 2, 1, 2, 2, 2, 1,
       1, 1, 2, 2, 0, 2, 1, 1, 0, 1, 1, 1, 2, 0, 0, 1, 0, 2, 1, 0, 1, 2,
       1, 2, 1, 1, 1, 0, 2, 0, 1, 1, 2, 2, 1, 2, 2, 1, 0, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 0,
       1, 1, 2, 2, 0, 1, 1, 1, 1, 2, 2, 1, 1, 1, 0, 1, 1, 1, 2, 2, 2, 1,
       2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 0, 1, 1, 1, 2, 0, 2, 0, 0, 1, 2,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 2, 2,
       1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1,
       1, 0, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,

In [107]:
from sklearn.metrics import classification_report as clf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd

target_names=['negative','neutral','positive']

df_cm = pd.DataFrame(confusion_matrix(y_test_int, y_pred_int), columns=target_names, index=target_names)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
print(df_cm)
print("\n")
print("Accuracy: ",accuracy_score(y_test_int, y_pred_int))
print("\n","Report:")
print(clf(y_test_int, y_pred_int, target_names=target_names))

Predicted  negative  neutral  positive
Actual                                
negative         40       64        15
neutral          18      336        31
positive         23      128       120


Accuracy:  0.64

 Report:
              precision    recall  f1-score   support

    negative       0.49      0.34      0.40       119
     neutral       0.64      0.87      0.74       385
    positive       0.72      0.44      0.55       271

    accuracy                           0.64       775
   macro avg       0.62      0.55      0.56       775
weighted avg       0.64      0.64      0.62       775



#GRU

In [108]:
from keras.layers import GRU
model_GRU = Sequential()
model_GRU.add(Bidirectional(LSTM(10, dropout=0.5, recurrent_dropout=0.5, return_sequences=True), input_shape=(48, 3072), merge_mode='concat'))
#model_GRU.add(GRU(8, dropout=0.5, recurrent_dropout=0.5, return_sequences=True))
model_GRU.add(SeqSelfAttention(attention_activation='sigmoid'))
model_GRU.add(GRU(8, dropout=0.5, recurrent_dropout=0.5, return_sequences=False))
#model_GRU.add(Dense(8, activation='softmax'))
model_GRU.add(Dense(3, activation='softmax'))
model_GRU.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_GRU.fit(x=X_train_bert_padded.cpu().numpy(), 
              y=y_train, 
              validation_data=(X_test_bert_padded.cpu().numpy(), y_test),	
              batch_size=64, epochs=10, shuffle=True)

Epoch 1/10
49/49 [==============================] - 74s 1s/step - loss: 1.0217 - accuracy: 0.5056 - val_loss: 0.9781 - val_accuracy: 0.5523
Epoch 2/10
49/49 [==============================] - 63s 1s/step - loss: 0.9624 - accuracy: 0.5501 - val_loss: 0.9510 - val_accuracy: 0.5613
Epoch 3/10
49/49 [==============================] - 63s 1s/step - loss: 0.9180 - accuracy: 0.5885 - val_loss: 0.9060 - val_accuracy: 0.5961
Epoch 4/10
49/49 [==============================] - 63s 1s/step - loss: 0.8749 - accuracy: 0.6046 - val_loss: 0.9068 - val_accuracy: 0.5948
Epoch 5/10
49/49 [==============================] - 63s 1s/step - loss: 0.8335 - accuracy: 0.6287 - val_loss: 0.8597 - val_accuracy: 0.6052
Epoch 6/10
49/49 [==============================] - 63s 1s/step - loss: 0.7989 - accuracy: 0.6423 - val_loss: 0.8520 - val_accuracy: 0.6181
Epoch 7/10
49/49 [==============================] - 63s 1s/step - loss: 0.7646 - accuracy: 0.6607 - val_loss: 0.8459 - val_accuracy: 0.6232
Epoch 8/10
49/49 [==

In [109]:
model_GRU.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_6 (Bidirectio  (None, 48, 20)           246640    
 nal)                                                            
                                                                 
 seq_self_attention_1 (SeqSe  (None, 48, 20)           1345      
 lfAttention)                                                    
                                                                 
 gru (GRU)                   (None, 8)                 720       
                                                                 
 dense_5 (Dense)             (None, 3)                 27        
                                                                 
Total params: 248,732
Trainable params: 248,732
Non-trainable params: 0
_________________________________________________________________


In [110]:
model_GRU.evaluate(X_test_bert_padded.cpu().numpy(), y_test)[1]

25/25 [==============================] - 3s 111ms/step - loss: 0.8597 - accuracy: 0.6361


0.6361290216445923

In [ ]:
#from keras.models import load_model
#model_GRU.save('my_model_GRU.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model
# returns a compiled model
# identical to the previous one
#model_GRU = load_model('my_model_GRU.h5')


In [113]:
y_pred = []          

for tweet in X_test_bert_padded.cpu().numpy():
  y_pred.append(model_GRU.predict(np.array([tweet])))

#y_pred = np.array([i[0] for i in y_pred])

In [114]:
y_list= list(y_pred)

In [115]:
y_pred_int =[]
for i in range(len(y_list)):
  y_pred_int.append(maximum((list(list(y_list[i]))[0][0]),(list(list(y_list[i]))[0][1]),(list(list(y_list[i]))[0][2])))


In [116]:
y_pred = np.array(y_pred_int)

In [117]:
y_pred

array([1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1,
       0, 2, 1, 1, 0, 1, 2, 0, 1, 1, 1, 2, 1, 2, 1, 1, 0, 0, 1, 1, 1, 2,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2, 1, 2, 2, 1, 1, 2, 2, 1,
       1, 1, 2, 1, 0, 0, 1, 0, 0, 2, 1, 0, 2, 0, 1, 1, 0, 2, 1, 1, 1, 2,
       2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 0,
       1, 1, 2, 2, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 2, 1, 2, 2,
       2, 1, 1, 1, 0, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 0, 0, 2, 0, 2, 1, 0,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 2, 1,
       2, 2, 1, 0, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1, 1, 2, 2,
       1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 0, 1, 0, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1,

In [118]:
from sklearn.metrics import classification_report as clf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd

target_names=['negative','neutral','positive']

df_cm = pd.DataFrame(confusion_matrix(y_test_int, y_pred), columns=target_names, index=target_names)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
print(df_cm)
print("\n")
print("Accuracy: ",accuracy_score(y_test_int, y_pred))
print("\n","Report:")
print(clf(y_test_int, y_pred, target_names=target_names))

Predicted  negative  neutral  positive
Actual                                
negative         36       64        19
neutral          18      319        48
positive         17      116       138


Accuracy:  0.6361290322580645

 Report:
              precision    recall  f1-score   support

    negative       0.51      0.30      0.38       119
     neutral       0.64      0.83      0.72       385
    positive       0.67      0.51      0.58       271

    accuracy                           0.64       775
   macro avg       0.61      0.55      0.56       775
weighted avg       0.63      0.64      0.62       775



#BERT

In [21]:
!pip install pytorch-pretrained-bert pytorch-nlp

     |████████████████████████████████| 123 kB 5.5 MB/s 
     |████████████████████████████████| 90 kB 9.7 MB/s 
     |████████████████████████████████| 131 kB 56.1 MB/s 
     |████████████████████████████████| 8.1 MB 46.4 MB/s 
     |████████████████████████████████| 79 kB 5.7 MB/s 
     |████████████████████████████████| 138 kB 57.6 MB/s 
     |████████████████████████████████| 127 kB 57.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [18]:
# Create sentence and label lists
sentences = X_train

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
y_dct = {'0':0, '1':1,'2':2}

for i, sentiment in enumerate(y_train):
  y_train[i] = y_dct[sentiment]

import numpy as np 
labels = np.array(y_train)

In [19]:
labels

array([2, 1, 1, ..., 0, 2, 1])

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [21]:
for tweet, bert_tokens, input_id in zip(X_train[:5],tokenized_texts[:5],input_ids[:5]):
  print(tweet)
  print(bert_tokens)
  print(input_id)
  print("\n")

Pehli bar desh ki "AAN BAN SHAN"dikhth he , jispe hame "NAZ" he.salam to p.m.
['[CLS]', 'pe', '##hli', 'bar', 'des', '##h', 'ki', '"', 'aa', '##n', 'ban', 'shan', '"', 'di', '##kh', '##th', 'he', ',', 'ji', '##sp', '##e', 'ham', '##e', '"', 'na', '##z', '"', 'he', '.', 'sal', '##am', 'to', 'p', '.', 'm', '.', '[SEP]']
[101, 21877, 27766, 3347, 4078, 2232, 11382, 1000, 9779, 2078, 7221, 17137, 1000, 4487, 10023, 2705, 2002, 1010, 10147, 13102, 2063, 10654, 2063, 1000, 6583, 2480, 1000, 2002, 1012, 16183, 3286, 2000, 1052, 1012, 1049, 1012, 102]


Sir life me ek baar apko milna he aapko milna he
['[CLS]', 'sir', 'life', 'me', 'ek', 'ba', '##ar', 'ap', '##ko', 'mil', '##na', 'he', 'aa', '##p', '##ko', 'mil', '##na', 'he', '[SEP]']
[101, 2909, 2166, 2033, 23969, 8670, 2906, 9706, 3683, 23689, 2532, 2002, 9779, 2361, 3683, 23689, 2532, 2002, 102]


# 4 baje ge aaaj party abhi baki.wow
['[CLS]', '#', '4', 'ba', '##je', 'ge', 'aaa', '##j', 'party', 'ab', '##hi', 'ba', '##ki', '.', 'wow', '[SE

In [22]:
MAX_LEN = max([len(bert_tokens) for bert_tokens in tokenized_texts])
print(MAX_LEN)

387


In [23]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [24]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [25]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [26]:
print(train_inputs.shape)
print(validation_inputs.shape)
print(train_labels.shape)
print(validation_labels.shape)

(2792, 387)
(311, 387)
(2792,)
(311,)


In [27]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [28]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [29]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
#model.cuda()

In [30]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [31]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,lr=2e-5,warmup=.1)

t_total value of -1 results in schedule not being applied


In [32]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 10

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]